In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/Turing/TuringTutorials/tutorials/docs-17-which-sampler`


In [37]:
using Turing, DynamicPPL, LogDensityProblems, LogDensityProblemsAD, Random, LinearAlgebra, Plots #, StatsPlots
using MicroCanonicalHMC, Pathfinder

[ Info: Precompiling MicroCanonicalHMC [234d2aa0-2291-45f7-9047-6fa6f316b0a8]


In [3]:
m10 = 10*rand(10)
s10 = Vector(LinRange(1, 10, 10)).^2

m20 = 10*rand(20)
s20 = Vector(LinRange(1, 10, 20)).^2

m30 = 10*rand(30)
s30 = Vector(LinRange(1, 10, 30)).^2

m50 = 10*rand(50)
s50 = Vector(LinRange(1, 10, 50)).^2

m100 = 10*rand(100)
s100 = Vector(LinRange(1, 10, 100)).^2;


In [23]:
cov10 = Diagonal(s10)

10×10 Diagonal{Float64, Vector{Float64}}:
 1.0   ⋅    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅     ⋅      ⋅ 
  ⋅   4.0   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅     ⋅      ⋅ 
  ⋅    ⋅   9.0    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅      ⋅ 
  ⋅    ⋅    ⋅   16.0    ⋅     ⋅     ⋅     ⋅     ⋅      ⋅ 
  ⋅    ⋅    ⋅     ⋅   25.0    ⋅     ⋅     ⋅     ⋅      ⋅ 
  ⋅    ⋅    ⋅     ⋅     ⋅   36.0    ⋅     ⋅     ⋅      ⋅ 
  ⋅    ⋅    ⋅     ⋅     ⋅     ⋅   49.0    ⋅     ⋅      ⋅ 
  ⋅    ⋅    ⋅     ⋅     ⋅     ⋅     ⋅   64.0    ⋅      ⋅ 
  ⋅    ⋅    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅   81.0     ⋅ 
  ⋅    ⋅    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅     ⋅   100.0

In [31]:
@model function Gaussian_model(data, cov)
    θ ~ filldist(Uniform(0, 10), length(data))
    data ~ MvNormal(θ, cov)
end

Gaussian_model (generic function with 2 methods)

In [32]:
Gaussian10 = Gaussian_model(m10, cov10)
Gaussian20 = Gaussian_model(m20, s20)
Gaussian30 = Gaussian_model(m30, s30)
Gaussian50 = Gaussian_model(m50, s50)
Gaussian100 = Gaussian_model(m100, s100)

Model{typeof(Gaussian_model), (:data, :cov), (), (), Tuple{Vector{Float64}, Vector{Float64}}, Tuple{}, DefaultContext}(Gaussian_model, (data = [1.6695718918468128, 4.585507840758493, 5.767730481311855, 5.706160066539352, 1.3780870302035464, 5.302116640382336, 2.4340180296591054, 5.8697114415739176, 8.14460635377149, 2.8025843589949586  …  8.699739679051982, 9.709856693201765, 5.923956270486373, 5.974974890052499, 8.005114707122106, 4.179710406287368, 0.43439562988778935, 6.234817251893442, 9.367419729431957, 3.911872622401683], cov = [1.0, 1.1900826446280994, 1.3966942148760333, 1.6198347107438016, 1.8595041322314052, 2.115702479338843, 2.3884297520661155, 2.6776859504132227, 2.9834710743801662, 3.3057851239669427  …  84.30578512396694, 85.98347107438018, 87.67768595041322, 89.38842975206613, 91.11570247933884, 92.85950413223138, 94.61983471074379, 96.39669421487604, 98.19008264462812, 100.0]), NamedTuple(), DefaultContext())

In [35]:
MH_chain_10 = sample(Gaussian10, MH(), 10_000)

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


Chains MCMC chain (10000×11×1 Array{Float64, 3}):

Iterations        = 1:1:10000
Number of chains  = 1
Samples per chain = 10000
Wall duration     = 0.17 seconds
Compute duration  = 0.17 seconds
parameters        = θ[1], θ[2], θ[3], θ[4], θ[5], θ[6], θ[7], θ[8], θ[9], θ[10]
internals         = lp

Summary Statistics
  parameters      mean       std      mcse   ess_bulk   ess_tail      rhat   e ⋯
      Symbol   Float64   Float64   Float64    Float64    Float64   Float64     ⋯

        θ[1]    8.7809    0.7654    0.0400   371.1069   353.0633    1.0049     ⋯
        θ[2]    1.7842    1.2683    0.0703   303.9006   317.9348    1.0005     ⋯
        θ[3]    6.1093    2.2095    0.1151   379.2343   417.5878    1.0018     ⋯
        θ[4]    5.8843    2.5130    0.1252   411.9949   272.5686    1.0128     ⋯
        θ[5]    4.6771    2.8109    0.1630   301.9897   165.9501    1.0088     ⋯
        θ[6]    4.7092    2.7379    0.1526   332.2990   322.1669    1.0001     ⋯
        θ[7]    5.1271    2.7533 

In [36]:
NUTS_chain_10 = sample(Gaussian10, NUTS(0.65), 10_000)

┌ Info: Found initial step size
└   ϵ = 0.8
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:01


Chains MCMC chain (10000×22×1 Array{Float64, 3}):

Iterations        = 1001:1:11000
Number of chains  = 1
Samples per chain = 10000
Wall duration     = 1.11 seconds
Compute duration  = 1.11 seconds
parameters        = θ[1], θ[2], θ[3], θ[4], θ[5], θ[6], θ[7], θ[8], θ[9], θ[10]
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std      mcse     ess_bulk    ess_tail      rhat  ⋯
      Symbol   Float64   Float64   Float64      Float64     Float64   Float64  ⋯

        θ[1]    8.8122    0.7589    0.0076    8175.0180   4897.3182    0.9999  ⋯
        θ[2]    1.7870    1.3016    0.0114    9868.3216   4841.3122    1.0002  ⋯
        θ[3]    5.9965    2.3256    0.0194   13888.1780   5647.4495    1.0005  ⋯
        θ[4]    5.8324    2.4927    0.0212   13328.7527   5854.8001    0.9999  ⋯
        θ[5

In [39]:
MCHMC_chain_10 = sample(Gaussian10, externalsampler(MCHMC(1_000, 0.01; adaptive=true)), 10_000)

[ Info: Tuning sigma ⏳
[ Info: Tuning eps ⏳
[ Info: Tuning L ⏳
2mMCHMC (tuning): 100%|███████████████████████████████████| Time: 0:00:00[ Info: eps: 1.5445883121321577
[ Info: L: 2.439300465707851
[ Info: nu: 0.5410277436142356
[ Info: sigma: [1.0588941461273873, 1.3118900954724115, 1.341765508460827, 1.46998753746638, 1.6215943083158675, 1.523626850951299, 1.8150543525404614, 1.4977411239671428, 1.9617863608142907, 1.9492879336661693]
[ Info: adaptive: true
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


Chains MCMC chain (10000×11×1 Array{Float64, 3}):

Iterations        = 1:1:10000
Number of chains  = 1
Samples per chain = 10000
Wall duration     = 1.91 seconds
Compute duration  = 1.91 seconds
parameters        = θ[1], θ[2], θ[3], θ[4], θ[5], θ[6], θ[7], θ[8], θ[9], θ[10]
internals         = lp

Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

        θ[1]    8.8032    0.8195    0.0145   2093.8771   1635.7433    1.0005   ⋯
        θ[2]    1.8160    1.3681    0.0236   2378.6185   1449.9089    1.0001   ⋯
        θ[3]    5.9793    2.3453    0.0494   1963.3934   1737.4542    1.0000   ⋯
        θ[4]    5.7732    2.5593    0.0550   1922.0678   1934.7374    1.0008   ⋯
        θ[5]    4.6775    2.6982    0.0562   2046.4492   2359.7564    1.0036   ⋯
        θ[6]    4.6191    2.7597    0.0647   1584.3593   1777.1165    1.0022   ⋯
        θ[7]    5.0999    2.8536 

In [55]:
draws = 1_000
pathfinder_10 = multipathfinder(Gaussian10, draws; nruns=8)


┌ Warning: Pareto shape k = 1 > 1. Corresponding importance sampling estimates are likely to be unstable and are unlikely to converge with additional samples.
└ @ PSIS ~/.julia/packages/PSIS/f7yyt/src/core.jl:290


Multi-path Pathfinder result
  runs: 8
  draws: 1000
  Pareto shape diagnostic: 1.04 (very bad)

In [61]:
sqrt.(diag(pathfinder_10.pathfinder_results[1].fit_distribution.Σ))

10-element Vector{Float64}:
 3.425720256429995
 2.7493424912932487
 1.3491869629302082
 1.9297810948789416
 2.395450489340556
 2.935300678879735
 2.815960902470621
 4.350325688971036
 4.0948434480854505
 4.512009253079756

In [9]:
ms = [mean(MH_chain_10.value.data[:, i]) for i in 1:10]
ss = [std(MH_chain_10.value.data[:, i]) for i in 1:10]

10-element Vector{Float64}:
 0.7689168071183567
 1.3300481145511651
 2.326394088205585
 2.5609213029547653
 2.612306847610716
 2.8195582127218906
 2.7327773318110173
 2.8579052426246827
 2.9228661776173563
 2.8754236326448743

In [10]:
plot(mean(MH_chain_10) .- m10)

LoadError: MethodError: no method matching length(::ChainDataFrame{NamedTuple{(:parameters, :mean), Tuple{Vector{Symbol}, Vector{Float64}}}})

[0mClosest candidates are:
[0m  length([91m::Union{Base.KeySet, Base.ValueIterator}[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mabstractdict.jl:58[24m[39m
[0m  length([91m::Union{SparseArrays.FixedSparseVector{Tv, Ti}, SparseArrays.SparseVector{Tv, Ti}} where {Tv, Ti}[39m)
[0m[90m   @[39m [35mSparseArrays[39m [90m~/.julia/juliaup/julia-1.9.4+0.x64.linux.gnu/share/julia/stdlib/v1.9/SparseArrays/src/[39m[90m[4msparsevector.jl:95[24m[39m
[0m  length([91m::Union{ZMQ._Message, Base.RefValue{ZMQ._Message}}[39m)
[0m[90m   @[39m [36mZMQ[39m [90m~/.julia/packages/ZMQ/lrABE/src/[39m[90m[4m_message.jl:31[24m[39m
[0m  ...


In [ ]:
MH_chain_20 = sample(Gaussian20, MH(), 10_000)

In [ ]:
MH_chain_30 = sample(Gaussian30, MH(), 10_000)

In [ ]:
MH_chain_50 = sample(Gaussian50, MH(), 10_000)

In [ ]:
MH_chain_100 = sample(Gaussian100, MH(), 10_000)

In [ ]:
ctxt = Gaussiann10.context
vi =  DynamicPPL.VarInfo(Gaussiann10, ctxt)

In [ ]:
vi_t = Turing.link!!(vi, Gaussiann10)
ℓ = LogDensityProblemsAD.ADgradient(DynamicPPL.LogDensityFunction(Gaussiann10, vi_t))
ℓπ(x) = LogDensityProblems.logdensity(ℓ, x)
∂lπ∂θ(x) = LogDensityProblems.logdensity_and_gradient(ℓ, x)

In [ ]:
ℓπ(3*Vector(1:10))

In [ ]:
ctxt = model.context
vi = DynamicPPL.VarInfo(model, ctxt)
vi_t = Turing.link!!(vi, model)
dists = _get_dists(vi)
dist_lengths = [length(dist) for dist in dists]
vsyms = _name_variables(vi, dist_lengths)
d = length(vsyms)

ℓ = LogDensityProblemsAD.ADgradient(DynamicPPL.LogDensityFunction(vi_t, model, ctxt))
ℓπ(x) = LogDensityProblems.logdensity(ℓ, x)
∂lπ∂θ(x) = LogDensityProblems.logdensity_and_gradient(ℓ, x)

In [ ]:
Gaussiann10

In [ ]:
plot(chain)